<a href="https://colab.research.google.com/github/avvni/Vehicle-Count-Prediction/blob/main/Vehicle_Count_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
uploaded=  files.upload()

Saving vehicles.csv to vehicles.csv


In [3]:
import pandas as pd
train = pd.read_csv('/content/vehicles.csv')
train.head()

,DateTime,Vehicles
0,2015-11-01 00:00:00,15
1,2015-11-01 01:00:00,13
2,2015-11-01 02:00:00,10
3,2015-11-01 03:00:00,7
4,2015-11-01 04:00:00,9


In [4]:
def get_dom(dt):
	return dt.day
def get_weekday(dt):
	return dt.weekday()
def get_hour(dt):
	return dt.hour
def get_year(dt):
	return dt.year
def get_month(dt):
	return dt.month
def get_dayofyear(dt):
	return dt.dayofyear
def get_weekofyear(dt):
	return dt.weekofyear

train['DateTime'] = train['DateTime'].map(pd.to_datetime)
train['date'] = train['DateTime'].map(get_dom)
train['weekday'] = train['DateTime'].map(get_weekday)
train['hour'] = train['DateTime'].map(get_hour)
train['month'] = train['DateTime'].map(get_month)
train['year'] = train['DateTime'].map(get_year)
train['dayofyear'] = train['DateTime'].map(get_dayofyear)
train['weekofyear'] = train['DateTime'].map(get_weekofyear)

train.head()

,DateTime,Vehicles,date,weekday,hour,month,year,dayofyear,weekofyear
0,2015-11-01 00:00:00,15,1,6,0,11,2015,305,44
1,2015-11-01 01:00:00,13,1,6,1,11,2015,305,44
2,2015-11-01 02:00:00,10,1,6,2,11,2015,305,44
3,2015-11-01 03:00:00,7,1,6,3,11,2015,305,44
4,2015-11-01 04:00:00,9,1,6,4,11,2015,305,44


In [5]:
train = train.drop(['DateTime'], axis=1)
train1 = train.drop(['Vehicles'], axis=1)
target = train['Vehicles']
print(train1.head())
target.head()


   date  weekday  hour  month  year  dayofyear  weekofyear
0     1        6     0     11  2015        305          44
1     1        6     1     11  2015        305          44
2     1        6     2     11  2015        305          44
3     1        6     3     11  2015        305          44
4     1        6     4     11  2015        305          44


,Vehicles
0,15
1,13
2,10
3,7
4,9


In [6]:
from sklearn.ensemble import RandomForestRegressor
m1=RandomForestRegressor()
m1.fit(train1,target)
m1.predict([[11,6,0,1,2015,11,2]])

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


array([9.36957143])

**EVALUATING MODEL PERFORMANCE AND ERROR ANALYSIS**

In [11]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np
X = train.drop('Vehicles', axis=1)
y = train['Vehicles']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

m1_evaluated = RandomForestRegressor(random_state=42)
m1_evaluated.fit(X_train, y_train)

y_pred = m1_evaluated.predict(X_test)

mse = mean_squared_error(y_test, y_pred)

print(f"Mean Squared Error (MSE): {mse}")


Mean Squared Error (MSE): 602.4201459337917


In [13]:
import numpy as np
rmse = np.sqrt(602.4201459337917)
print(rmse)


24.544248734353058


In [14]:
from sklearn.metrics import mean_squared_error

# Baseline: Predict mean of y for all
y_baseline = [y_train.mean()] * len(y_test)
baseline_mse = mean_squared_error(y_test, y_baseline)

print(f"Baseline MSE: {baseline_mse}")
print(f"Model MSE: {mse}")


Baseline MSE: 407.75956068783194
Model MSE: 602.4201459337917


The current MSE is much higher than the Baseline MSE. Hence, tuning the model further to ensure an accurate model.

**TUNING MODEL**


In [16]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

X = train.drop('Vehicles', axis=1)
y = train['Vehicles']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5],
    'max_features': ['sqrt', 'log2']
}

rf = RandomForestRegressor(random_state=42)
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    scoring='neg_mean_squared_error',
    cv=3,
    n_jobs=-1,
    verbose=2
)


grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

print("Best Parameters Found:", best_params)

# Predict and calculate MSE
y_pred_best = best_model.predict(X_test)
mse_best = mean_squared_error(y_test, y_pred_best)
rmse_best = np.sqrt(mse_best)

print(f" Tuned Model MSE: {mse_best:.2f}")
print(f" Tuned Model RMSE: {rmse_best:.2f}")


baseline_prediction = np.full_like(y_test, y_train.mean())
baseline_mse = mean_squared_error(y_test, baseline_prediction)
improvement = (baseline_mse - mse_best) / baseline_mse * 100

print(f" Baseline MSE: {baseline_mse:.2f}")
print(f" Improvement over baseline: {improvement:.2f}%")


Fitting 3 folds for each of 24 candidates, totalling 72 fits
Best Parameters Found: {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_split': 5, 'n_estimators': 200}
 Tuned Model MSE: 346.85
 Tuned Model RMSE: 18.62
 Baseline MSE: 407.73
 Improvement over baseline: 14.93%
